To install rascal:
(NOTE: See the top-level README for the most up-to-date installation instructions.)
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

In [ ]:
!export OMP_NUM_THREADS=1
from mkl import set_num_threads
set_num_threads(1)

In [ ]:
%matplotlib inline
from matplotlib import pylab as plt

import os, sys
from ase.io import read
sys.path.insert(0,"../build/")

import sys
import time
import rascal
import json

import ase
from ase.io import read, write
from ase.build import make_supercell
from ase.visualize import view
import numpy as np
import sys

import json

from rascal.representations import SphericalInvariants
from rascal.models import Kernel
from rascal.utils import CURFilter

In [ ]:
def json_writer(fn, frames):
    import json
    frame_dict = {}
    for i_frame, ff in enumerate(frames):
        frame_dict[str(i_frame)] = {"cell":ff.cell.tolist(), "positions":ff.positions.tolist(), 
                                    "pbc" : ff.pbc.tolist(), "numbers":ff.numbers.tolist()}
    frame_dict["ids"] = list(range(len(frames)))
    frame_dict["nextid"] =  len(frames)+1
    with open(fn, 'w') as f:
        json.dump(frame_dict,f,indent=2)

# test computing sparse kernel gradient with finite differences

In [ ]:
frames = read('../../replicated_input.xyz',':5')
write('../reference_data/inputs/ni.json', frames)

In [ ]:
# Load the small molecules 
# frames = read('../reference_data/inputs/small_molecules-20.json',':10')
# frames = read('../reference_data/inputs/SiC_moissanite_supercell.json',':')

In [ ]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=3, 
              max_angular=3, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)

# Select pseudo input with CUR decomposition
# n_pseudo = {1:5,6:5,8:5,7:5}
# n_pseudo = {6:7,14:7}
n_pseudo = {28:5}
compressor = CURFilter(soap, n_pseudo, act_on='sample per specie')

X_pseudo = compressor.fit_transform(managers)

In [ ]:
str(compressor.selected_ids).replace('[','{').replace(']','}')
compressor.selected_ids

In [ ]:
hypers['compute_gradients'] =  True
soap = SphericalInvariants(**hypers)
zeta = 1
kernel = Kernel(soap,name='GAP', zeta=zeta, target_type='Structure', kernel_type='Sparse')

In [ ]:
# compute the reference
managers = soap.transform([frames[0]])
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(True, False))

In [ ]:
# test for one row
def update_frame(frame, i_at, disp):
    ff = frame.copy()
    ff.positions[i_at] += np.asarray(disp)
    return ff

def compute_displaced_kernel(kernel, rep, frame, i_atom, disp, X_pseudo):
    frame_d = update_frame(frame, i_atom, disp)
    managers = rep.transform([frame_d])
    KNM_d = kernel(managers, X_pseudo, compute_gradients=(False, False))
    return KNM_d


frame = frames[0]
eps = 1e-5
i_atom = 0
disps = eps*np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1],])
kernel = Kernel(soap,name='GAP', zeta=zeta, target_type='Atom', kernel_type='Sparse')

KNM_p = compute_displaced_kernel(kernel, soap, frame, i_atom, disps[0],X_pseudo)
KNM_m = compute_displaced_kernel(kernel, soap, frame, i_atom, disps[1],X_pseudo)
KNM_r_x = (KNM_p - KNM_m).sum(axis=0)  / (2*eps)
np.allclose(KNM_r_x,KNM_ref[0])

In [ ]:
# test for a whole structure
def compute_numerical_kernel_gradient(kernel, representation, frame, X_pseudo, eps=1e-5):
    disps = eps*np.array([[[1,0,0],[-1,0,0]],[[0,1,0],[0,-1,0]],[[0,0,1],[0,0,-1]]])
    KNM_der = np.zeros((len(frame)*3, X_pseudo.size()))

    for i_atom in range(len(frame)):
        for i_der, disp in enumerate(disps):
            KNM_p = compute_displaced_kernel(kernel, representation, frame, i_atom, disp[0], X_pseudo)
            KNM_m = compute_displaced_kernel(kernel, representation, frame, i_atom, disp[1], X_pseudo)
            KNM_der[i_atom*3+i_der] = (KNM_p - KNM_m).sum(axis=0) / (2*eps)
    return KNM_der

KNM_test = compute_numerical_kernel_gradient(kernel, soap, frame, X_pseudo)
np.allclose(KNM_test,KNM_ref)

In [ ]:
KNM_ref[0]

In [ ]:
KNM_test[0]

In [ ]:
np.vstack([KNM_test,KNM_ref]).shape,KNM_ref.shape

# test molecules

In [ ]:
# Load the small molecules 
frames = read('../reference_data/inputs/small_molecules-20.json',':5')
n_pseudo = {1:3,6:3,8:3,7:3}

hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=3, 
              max_angular=3, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)


compressor = CURFilter(soap, n_pseudo, act_on='sample per species')

X_pseudo = compressor.fit_transform(managers)

hypers['compute_gradients'] =  True
soap = SphericalInvariants(**hypers)

kernel = Kernel(soap, name='GAP', zeta=2, target_type='Atom', kernel_type='Sparse')

managers = soap.transform(frames)
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(True, False))
KNM_test = kernel(frames, X_pseudo, compute_gradients=(True, False), numerical_gradients=True)

print(compressor.selected_ids)
print(np.allclose(KNM_test,KNM_ref,rtol=1e-10,atol=1e-13))
print(KNM_ref[0])
print(KNM_test[0])

# test diamond

In [ ]:
frames = read('../reference_data/inputs/diamond_cubic_distorted.json',':')
n_pseudo = {6:5}


hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3, 
              max_radial=3, 
              max_angular=3, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)


compressor = CURFilter(soap, n_pseudo, act_on='sample per species')

X_pseudo = compressor.fit_transform(managers)

hypers['compute_gradients'] =  True
soap = SphericalInvariants(**hypers)

kernel = Kernel(soap, name='GAP', zeta=1, target_type='Atom', kernel_type='Sparse')

managers = soap.transform(frames)
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(True, False))
KNM_test = kernel(frames, X_pseudo, compute_gradients=(True, False), numerical_gradients=True)

print(compressor.selected_ids)
print(np.allclose(KNM_test,KNM_ref,rtol=1e-3,atol=1e-13))
print(KNM_ref[0])
print(KNM_test[0])

In [ ]:
-7.76335e-07  -7.5826e-07 -7.76335e-07 -7.59014e-07 -7.56195e-07

In [ ]:
from ase.neighborlist import neighbor_list

frame = read('../reference_data/inputs/diamond_cubic_distorted.json')
i, j, d, D, S = neighbor_list('ijdDS', frame, 3.5)

In [ ]:
frame.get_cell_lengths_and_angles()

In [ ]:
i

In [ ]:
np.sort(d[i==0])

In [ ]:
ase.Atom?

In [ ]:
ff.set_atomic_numbers

In [ ]:
ff = frame.copy()
i_atom = 0
i, j, d, D, S = neighbor_list('ijdDS', ff, 5)
pp = ff.get_positions()
ff.set_atomic_numbers(np.ones(len(ff)))
ff.append(ase.Atom(position=pp[i_atom],symbol='C'))
for jj,SS in zip(j[i==i_atom],S[i==i_atom]):
    if jj == 5:
        ff.append(ase.Atom(position=pp[jj]+SS.dot(ff.cell),symbol='C'))
view(ff)

In [ ]:
j[i == 0]

In [ ]:
np.bincount(j[i == 0])

In [ ]:
S[i == 0]

In [ ]:
S[i == 0]

In [ ]:
neighbor_list?

# test ni

In [ ]:
frames = read('../../replicated_input.xyz',':5')
json_writer('../reference_data/inputs/ni.json',frames)

In [ ]:
frames = read('../reference_data/inputs/ni.json',':4')
n_pseudo = {28:5}


hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=5, 
              max_radial=3, 
              max_angular=3, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)


compressor = CURFilter(soap, n_pseudo, act_on='sample per species')

X_pseudo = compressor.fit_transform(managers)

hypers['compute_gradients'] =  True
soap = SphericalInvariants(**hypers)

kernel = Kernel(soap, name='GAP', zeta=2, target_type='Atom', kernel_type='Sparse')

managers = soap.transform(frames)
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(True, False))
KNM_test = kernel(frames, X_pseudo, compute_gradients=(True, False), numerical_gradients=True)

print(compressor.selected_ids)
print(np.allclose(KNM_test,KNM_ref,rtol=1e-10,atol=1e-13))
print(KNM_ref[0])
print(KNM_test[0])

In [ ]:
print(KNM_ref[559])
print(KNM_test[559])

# test ni-h

In [ ]:
frames = read('../reference_data/inputs/ni.json',':5')
for ff in frames:
    ff.numbers[::2] = 1
json_writer('../reference_data/inputs/ni-h.json',frames)

In [ ]:
frames = read('../reference_data/inputs/ni-h.json',':5')
n_pseudo = {28:5,1:5}


hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=5, 
              max_radial=3, 
              max_angular=3, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)


compressor = CURFilter(soap, n_pseudo, act_on='sample per species')

X_pseudo = compressor.fit_transform(managers)
print(compressor.selected_ids)
hypers['compute_gradients'] =  True
soap = SphericalInvariants(**hypers)

kernel = Kernel(soap, name='GAP', zeta=2, target_type='Atom', kernel_type='Sparse')

managers = soap.transform(frames)
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(True, False))
KNM_test = kernel(frames, X_pseudo, compute_gradients=(True, False), numerical_gradients=True)


print(np.allclose(KNM_test,KNM_ref))
print(KNM_ref[0])
print(KNM_test[0])

# test the rascal infrastructure

In [ ]:
# Load the small molecules 
frames = read('../reference_data/inputs/small_molecules-1000.xyz',':100')

In [ ]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=3, 
              max_angular=3, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)

# Select pseudo input with CUR decomposition
n_pseudo = {1:10,6:10,8:10,7:10}

compressor = CURFilter(soap, n_pseudo, act_on='sample per specie')

X_pseudo = compressor.fit_transform(managers)

In [ ]:
hypers['compute_gradients'] =  True
soap = SphericalInvariants(**hypers)

kernel = Kernel(soap, name='GAP', zeta=1, target_type='Structure', kernel_type='Sparse')

In [ ]:
%%time
# compute the reference
managers = soap.transform(frames)
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(False, False))

In [ ]:
%%time
# compute the reference
managers = soap.transform(frames)
KNM_ref = kernel(managers, X_pseudo, compute_gradients=(True, False))

In [ ]:
%%time
KNM_test = kernel(frames, X_pseudo, compute_gradients=(True, False), numerical_gradients=True)
np.allclose(KNM_test,KNM_ref)

In [ ]:
# the numerical gradients are quite slow to compute so here is a way to speed it up
# the kernel used has to be already in the namespace and in variable 'kernel'
from concurrent.futures import as_completed, ProcessPoolExecutor, ThreadPoolExecutor
from tqdm.notebook import tqdm

def get_strides(frames):
    Nstructures = len(frames)
    Ngrad_stride = [0]
    Ngrads = 0
    for frame in frames:
        n_at = len(frame)
        Ngrad_stride.append(n_at*3)
        Ngrads += n_at*3
    Ngrad_stride = np.cumsum(Ngrad_stride)
    return Nstructures,Ngrads,Ngrad_stride

def compute_grad(i_frame, frame):
    feat = kernel._rep.transform([frame])
    grad_rows = kernel(feat, X_pseudo, grad=(True, False))
    return grad_rows

def compute_grad_numerical(i_frame,frame):
    grad_rows = kernel([frame], X_pseudo, grad=(True, False), numerical_grad=True)
    return grad_rows

def compute_KNM(frames,max_workers=20, numerical_gradients=False):    
    Nstructures,Ngrads,Ngrad_stride = get_strides(frames)
    KNM = np.zeros((Ngrads, X_pseudo.size()))
    pbar = tqdm(frames,desc='kernel',leave=False)
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        if not numerical_grad:
            future_to_compute = {executor.submit(compute_grad,i_frame, frame):i_frame
                                                     for i_frame,frame in enumerate(frames)}
        else:
            future_to_compute = {executor.submit(compute_grad_numerical,i_frame, frame):i_frame
                                                     for i_frame,frame in enumerate(frames)}
        for future in as_completed(future_to_compute):
            i_frame = future_to_compute[future]
            grad_rows = future.result()
            KNM[Ngrad_stride[i_frame]:Ngrad_stride[i_frame+1]] = grad_rows
            pbar.update()
    return KNM


In [ ]:
%%time
kk = compute_KNM(frames, max_workers=1, numerical_gradients=False)
np.allclose(kk,KNM_ref)

In [ ]:
%%time
kk = compute_KNM(frames, max_workers=4, numerical_gradients=True)
np.allclose(kk,KNM_ref)

# Compute derivatives with the bispectrum

In [ ]:
# Load the small molecules 
frames = read('../reference_data/inputs/small_molecules-1000.xyz',':100')

In [ ]:
hypers = dict(soap_type="BiSpectrum",
              interaction_cutoff=3.5, 
              max_radial=2, 
              max_angular=2, 
              gaussian_sigma_constant=0.5,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=False,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)

managers = soap.transform(frames)

# Select pseudo input with CUR decomposition
n_pseudo = {1:10,6:10,8:10,7:10}

compressor = CURFilter(soap, n_pseudo, act_on='sample per specie')

X_pseudo = compressor.fit_transform(managers)

In [ ]:
hypers['compute_gradients'] =  False
soap = SphericalInvariants(**hypers)

kernel = Kernel(soap, name='GAP', zeta=1, target_type='Structure', kernel_type='Sparse')

In [ ]:
%%time
KNM = kernel(frames, X_pseudo, compute_gradients=(True, False), numerical_gradients=True)